# Importing the libraries

In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow

# Setting up the required paths

In [ ]:
import os
print(os.listdir("../input/chest-xray-pneumonia/chest_xray"))
print(os.listdir("../input/chest-xray-pneumonia/chest_xray/val"))
print(os.listdir("../input/chest-xray-pneumonia/chest_xray/test"))
print(os.listdir("../input/chest-xray-pneumonia/chest_xray/train"))

In [ ]:
#setting trainning and validation path in the extracted folder
TRAIN_PATH="../input/chest-xray-pneumonia/chest_xray/train"
VAL_PATH="../input/chest-xray-pneumonia/chest_xray/val"
TEST_PATH="../input/chest-xray-pneumonia/chest_xray/test"

# Creating our own CNN model

In [ ]:
#CNN model in Keras
#Sequential model
#layered architecture
#we are creating 3-4 CNN layers with some classification layers
#first we put a convolutional layer with 32 filers
#adding two layers increases the non-linearity and reduce the number of parameters
#as we go deeper into the network we increase the number of filters because as we go deeper the numeber of distinct patterns we see are more


model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))  #single neuron
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

# Training the model from scratch

In [ ]:
#training from scratch
#we use keras image generator library
#we can't do vertical flip as all the x-ray should have same orientation

train_datagen= image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

test_dataset=image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'binary'
)

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    "../input/chest-xray-pneumonia/chest_xray/val",
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'binary')

In [ ]:
validation_generator.class_indices

In [ ]:
nb_train_samples = 5216
nb_validation_samples = 16
batch_size=16

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // (batch_size*4),
    epochs = 8,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
scores = model.evaluate_generator(validation_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Saving the model for future use

In [ ]:
model.save('model_p.h5')

# Plotting the accuracy curves

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.legend(['validation'], loc='upper left')
plt.show()


In [ ]:
#test images
model=load_model("model_p.h5")

In [ ]:
train_generator.class_indices

In [ ]:
y_actual=[]
y_test=[]

In [ ]:
import os
for i in os.listdir("../input/chest-xray-pneumonia/chest_xray/val/NORMAL/"):
  img=image.load_img("../input/chest-xray-pneumonia/chest_xray/val/NORMAL/"+i,target_size=(224,224))
  img=image.img_to_array(img)
  img=np.expand_dims(img, axis=0)
  p=model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(1)

In [ ]:
for i in os.listdir("../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/"):
  img=image.load_img("../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/"+i,target_size=(224,224))
  img=image.img_to_array(img)
  img=np.expand_dims(img, axis=0)
  p=model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(0)

In [ ]:
y_actual=np.array(y_actual)
y_test=np.array(y_test)

# Creating a confusion matrix for better visualization

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_actual,y_test)

In [ ]:
import seaborn as sns
sns.heatmap(cm, cmap="plasma", annot=True)